<a href="https://colab.research.google.com/github/chrober24/SL_2021-2022/blob/main/stylegan2_tutorial_generation_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stylegan2: Generating images
This will install all the necessary libraries to use the StyleGAN2 repo. Press the play button or `shift+return` to run each cell.

Only run the next cell once per session.


# setup

In [1]:
#@title Set tensorflow version
#@markdown always use tensorflow1

%tensorflow_version 1.x


TensorFlow 1.x selected.


In [2]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Install StyleGAN2 Repository
#@markdown StyleGAN2 will be installed to your Google Drive to speed up the training process

#@markdown Run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.
import os
import shlex
!pip install opensimplex
if os.path.isdir("/content/drive/MyDrive/stylegan2-aug-colab"):
  %cd "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2"
elif os.path.isdir("/content/drive"):
  %cd /content/drive/MyDrive/
  !mkdir stylegan2-aug-colab
  %cd stylegan2-aug-colab/
  !git clone -b augs-attn https://github.com/dvschultz/stylegan2
  %cd stylegan2
  !mkdir raw_images
  !mkdir pkl
  %cd pkl
  !gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF #inception: https://drive.google.com/open?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
  %cd ../
  !mkdir results
  !mkdir results/00001-pretrained
  %cd results/00001-pretrained
  !gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
  %cd ../../
  %mkdir datasets
else:
  !git clone -b augs-attn https://github.com/dvschultz/stylegan2
  %cd styelgan2
  !mkdir raw_images
  !mkdir pkl
  %cd pkl
  !gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF #inception: https://drive.google.com/open?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
  %cd ../
  !mkdir results
  !mkdir results/00001-pretrained
  %cd results/00001-pretrained
  !gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
  %cd ../../
  %mkdir datasets

In [4]:
#@title Network Selection
#@markdown This is where you will choose the model you would like to generate images from.

#@markdown change the file path here to the pkl of your most recent training session found in "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/results/{model-name}" 
network = "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/results/00057-stylegan2-tech-1gpu-config-f/network-snapshot-010024.pkl" #@param {type: "string"}

# Seed Generation

##Options

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.



*   For single image, choose a single seed number: 23
*   For a range of seeds: `[first]`-`[last]` (1-10) 
*   For List of seeds: `seed1`,`seed2`,`seed3`,...`seedn` (1,12,34,125)
  * no spaces in any lists or ranges



`--truncation-psi`

Truncation is a special argument of StyleGAN. Essentially values that are closer to 0 will be more real than numbers further away from 0. I generally recommend a value between `0.5` and `1.0`. `0.5` will give you pretty "realistic" results, while `1.0` is likely to give you "weirder" results. Truncation is looked at more closely in the next section.

note: run !python run_generator.py -h  for a list of all possible options

In [ ]:
#@title Generate
seeds = "5" #@param {type: "string"}
truncation_psi = 0.7 #@param {type: "number"}
!python run_generator.py generate-images --network=$network --seeds=$seeds --truncation-psi=$truncation_psi

##zip the generated files and download them.

`zip_input`: path to the folder you would like to zip
`zip_output`: path to folder where the zip file will be **placed**

In [ ]:
#@title **zip files**
zip_input = "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/results/00090-generate-images/" #@param
zip_output = "/content/example.zip" #@param
!zip -r $zip_output $zip_input

In [ ]:
#@title **Download Zip**
from google.colab import files
files.download(zip_output)

#Near neighbors
**Added by popular demand**

Let’s say you have a seed you like, but want to see other images like it to see if there’s something better. Now you can with the `near-neighbor` argument.

### Options
`--network`, `--seeds`, and `--truncation_psi` work the same as above.

`--diameter`: this sets how far away from the seed you want to generate images. `.0000001` is really close, `.5` is reallly far.

`--num_samples`: how many samples you want to produce

`--save_vector`: this will save the vector as a file in the .npy format. You can then use this for interpolation (not super well supported right now, but can be used manually—see the Projection code for an example of reading a .npy file and interpolating it).




In [ ]:
#@title Generate Near Neighbors
seeds = "5" #@param {type:"string"}
truncation_psi = 0.5 #@param {type:"number"}
num_samples = 2 #@param {type: "number"}
diameter = 0.5 #@param {type: "number"}
!python run_generator.py generate-neighbors --network=$network --seeds=$seeds --truncation-psi=$truncation_psi --num_samples=$num_samples --diameter=$diameter --save_vector

# Linear Interpolation

Linear interpolation will generate a linear path from one seed to another. The makers of StyleGAN say that doing this in the w space produces the best disentangled interpolations. 

`walk_type` : can be line-z or line-w. Line-z is the standard; however, z-space is entangled so the output may not appear completely linear. w-space is disentangled and thus will produce more linear outputs. It is recommended to start with line-z

`--seeds`: Use images you generated to control the interpolation points. If your first and last seed are the same this will produce a loop (nice for Instagram and gifs!)

`frames`: The total number of frames to be generated. Seeds will be spread out evenly in frame range

`truncation_psi`: same as previous steps

In [ ]:
#@title Generate Linear Interpolation
walk_type = 'line-w' #@param ['line-z','line-w']
seeds = "1,20" #@param {type: "string"}
frames = 3 #@param {type: "number"}
trucation_psi = 0.7 #@param {type: "number"}

!python run_generator.py generate-latent-walk --network=$network --walk-type=$walk_type --seeds=$seeds --frames $frames --truncation-psi=$truncation_psi

 ## Generate a video from the frames

 `foldernum`: number of the folder created by the interpolation script. (Example: for folder "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/results/00097-generate-latent-walk" `foldernum`=00097)

`framerate`: framerate of video. Can be any number you'd like (24 is standard cinematic framerate)

`video_name`: name of the video to be created

videos can be found at "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/{yourvideo.mp4}"

In [ ]:
#@title Produce Video 
foldernum = "00097" #@param {type: "string"}
folderpath = "./results/" + foldernum + "-generate-latent-walk/frame%05d.png"
framerate = 24 #@param {type: "number"}
video_name = "interpolation.mp4" #@param {type: "string"}

!ffmpeg -r $framerate -i $folderpath -vcodec libx264 -pix_fmt yuv420p $video_name

# Truncation
Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. Most people choose between 0.5 and 1.0, but technically its infinite. 

Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

###Options 

`--seed`: Pass this only one seed. Pick a favorite from your generated images.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)

In [ ]:
#@title Generate Truncation Frames
seed = 1 #@param {type:"number"}
start = -1.0 #@param {type: "number"}
stop = 1.0 #@param {type: "number"}
increment = 0.05 #@param {type: "number"}

!python run_generator.py truncation-traversal --network=$network --seed=$seed --start=$start --stop=$stop --increment=$increment

In [ ]:
#@title Produce Truncation Video 
foldernum = "00097" #@param {type: "string"}
folderpath = "./results/" + foldernum + "-truncation-traversal/frame%05d.png"
framerate = 24 #@param {type: "number"}
video_name = "truncation.mp4" #@param {type: "string"}

!ffmpeg -r $framerate -i $folderpath -vcodec libx264 -pix_fmt yuv420p $video_name

#Projection
Projection is the process of taking an image from outside the model and finding it’s nearest representation inside the model.

In [ ]:
#@title Projection Functions
#@markdown This sets up all of the functions we need for projection

!gdown --id 1gbxwfHNOaGjGsLNTmmSrNA85X2VWHHOq -O /content/drive/MyDrive/stylegan2-aug-colab/stylegan2/pkl/vgg16_zhang_perceptual.pkl

network_pkl = network

import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
from io import BytesIO
import numpy as np
from math import ceil
from PIL import Image, ImageDraw
import imageio

import pretrained_networks

# Convert uploaded images to TFRecords
import dataset_tool

# Run the projector
import training.dataset
import training.misc
import projector
import os 

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

def interpolate(zs, steps,type='linear'):
  out = []
  for i in range(len(zs)-1):
    c = zs[i+1]-zs[i]

    for index in range(steps):
      fraction = index/float(steps) # t/d
      
      # translated from: https://github.com/danro/jquery-easing/blob/master/jquery.easing.js
      # see https://easings.net/ for examples
      if type == 'linear':
        out.append( c * fraction + zs[i] ) # c*(t/d)+b
      elif type == 'easeInSine':
        out.append( -c * np.cos(fraction * (np.pi/2)) + c + zs[i] ) # -c * Math.cos(t/d * (Math.PI/2)) + c + b
      elif type == 'easeOutSine':
        out.append( c * np.sin(fraction * (np.pi/2)) + zs[i]) # c * Math.sin(t/d * (Math.PI/2)) + b
      elif type == 'easeInOutSine':
        out.append(-c/2 * (np.cos(np.pi*fraction) - 1.0) + zs[i]) # -c/2 * (Math.cos(Math.PI*t/d) - 1) + b;
      elif type == 'easeInQuad':
        out.append(c * fraction * fraction + zs[i]) # c*(t/=d)*t + b;
      elif type == 'easeOutQuad':
       out.append(-c * fraction * (fraction-2) + zs[i]) # -c *(t/=d)*(t-2) + b;
      # elif type == 'easeInOutQuad':
      #   if(fraction/2 < 1):
      #     out.append( ((c/2)*fraction*fraction) + zs[i]) #if ((t/=d/2) < 1) return c/2*t*t + b;
      #   else:
		  #     out.append( (-c/2) * ((index-=1)*(index-2) - 1) + zs[i]; #return -c/2 * ((--t)*(t-2) - 1) + b;
      else: 
        out.append( c * fraction + zs[i] ) # c*(t/d)+b
  return out

def saveImgs(imgs, location):
  for idx, img in log_progress(enumerate(imgs), size = len(imgs), name="Saving images"):
    file = location + ('%05d.png' % (idx))
    img.save(file)

def generate_images_in_w_space(dlatents, truncation_psi):
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    Gs_kwargs.truncation_psi = truncation_psi
    dlatent_avg = Gs.get_var('dlatent_avg') # [component]

    imgs = []
    for row, dlatent in enumerate(dlatents):
        #row_dlatents = (dlatent[np.newaxis] - dlatent_avg) * np.reshape(truncation_psi, [-1, 1, 1]) + dlatent_avg
        dl = (dlatent-dlatent_avg)*truncation_psi   + dlatent_avg
        row_images = Gs.components.synthesis.run(dlatent,  **Gs_kwargs)
        imgs.append(PIL.Image.fromarray(row_images[0], 'RGB'))
    return imgs  

print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)
noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

In [94]:
#@title make projection directories
#@markdown Next, we need to make some folders so we can upload the image for projection. Only run this cell once per session.

!mkdir projection
!mkdir projection/imgs
!mkdir projection/out
!mkdir projection/records
!mkdir projection/latents

mkdir: cannot create directory ‘projection’: File exists
mkdir: cannot create directory ‘projection/imgs’: File exists
mkdir: cannot create directory ‘projection/out’: File exists
mkdir: cannot create directory ‘projection/records’: File exists
mkdir: cannot create directory ‘projection/latents’: File exists


In [ ]:
#@title Projection Image Upload

#@markdown Now upload a single image to stylegan2/projection/imgs (can also use the Files side panel). Image should be color PNG, with the same dimensions as your dataset.

#@markdown run this cell twice to upload two images
#change into imgs folder
%cd ./projection/imgs

#user input file uploader
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
#change back to stylegan2 directory
%cd /content/drive/MyDrive/stylegan2-aug-colab/stylegan2

In [ ]:
#@title convert images into tfrecords

#@markdown This will convert the images you just uploaded into tfrecords for projecting into the model


#@markdown do not change any filepaths in this script
!rm ./projection/records/*.*
!python dataset_tool.py create_from_images_raw /content/drive/MyDrive/stylegan2-aug-colab/stylegan2/projection/records/ProjectFiles /content/drive/MyDrive/stylegan2-aug-colab/stylegan2/projection/imgs --result-dir= /content/drive/MyDrive/stylegan2-aug-colab/stylegan2/projection/out

### Project Options



*  `num_images`: the number of images you want to project (same as number of images in the ./projection/imgs/ folder)
*   `num_snapshots`: number of intermediate steps to save (max: 1000)





In [ ]:
#@title Project Images
!rm ./projection/*.*

num_imaages = 2 #@param {type: "number"}
num_snapshots = 5 #@param {type: "number"}
!python run_projector.py project-real-images --network=$network --dataset=ProjectFiles --data-dir=/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/projection/records --num-images=$num_images --num-snapshots=$num_snapshots --result-dir=/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/projection/out

### Make an interpolation!

This script will make a video interpolating between the two projected latent vectors from the last step.

`latent_path1,2`: file path to .npy files created in previous step. (found in ./projection/latents 

`frame_count`: total number of frames to create

`framerate`: same as previous steps

`video_name`: name of the output interpolation video

In [ ]:


latent_path1 = "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/projection/latent0.npy" #@param {type: "string"}
latent_path2 = "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/projection/latent1.npy" #@param {type: "string"}
latent1 = np.load(latent_path1)
latent2 = np.load(latent_path2)
frame_count = 144 #@param {type: "number"}

imgs = generate_images_in_w_space(interpolate([latent1,latent2],frame_count,'linear'),0.7)

!rm -rf interpolations
%mkdir interpolations
#save interpolation frames
saveImgs(imgs,'./interpolations/')

#make video
framerate = 24 #@param {type: "number"}
video_name = "projected_interpolation.mp4" #@param {type: "string"}
!ffmpeg -r $framerate -i ./interpolations/%05d.png -vcodec libx264 -pix_fmt yuv420p $video_name
